In [35]:
# WILL BE USED FOR GATHERING DATA
# requires older version of sklearn! 19.0 or older! (conda install scikit-learn==0.19)

import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi
from shapely.geometry import Polygon, Point
from scipy.cluster.vq import vq, kmeans, whiten

# get election result code from g6
import sys
import os
sys.path.append(os.getcwd())


#to make my plotting style
plt.rcParams["axes.facecolor"] = (1,1,.99) 
plt.rcParams["font.size"] = 15 
plt.rcParams["figure.figsize"] = (12,6) 
plt.rcParams["ytick.labelsize"] = 13 
plt.rcParams["xtick.labelsize"] = 13 
plt.rcParams["lines.linewidth"] = 2 
plt.rcParams["axes.titlesize"] = 17 

### Globals

In [36]:
NUM_PARTY_REPS = 3

### Run Vote

In [1]:
# Repurposed from group 6's voting program. Thanks group 6! 

def get_dist_results(dist_voters):
    # Get the voting results in each district (number of votes each party gets)
    # Step 1: Initialization
    n_parties = len(dist_voters[0][0]) - 2  # number of parties
    dist_results = {}
    dist_results[-1] = [0 for i in range(n_parties)]  # overall results
    for d in range(len(dist_voters)):
        dist_results[d] = [0 for i in range(n_parties)]  # results in each district
    # Step 2: Calculate results district by district
    for d in range(len(dist_voters)):
        for v in dist_voters[d]:
            this_pref = v[2:]  # party preferences for this voter
            this_party = np.argmax(this_pref)  # the party most preferred by this voter
            dist_results[-1][this_party] += 1
            dist_results[d][this_party] += 1
    return dist_results

def get_one_dist_seats(dist_votes, n_rep, reps_per_dist=3):
    # Get the number of seats for each party in one district
    # Step 1: Get the percentages of votes for each party
    n_parties = len(dist_votes)  # number of parties
    votes = [dist_votes[i] / sum(dist_votes) for i in range(n_parties)]
    # Step 2: Calculate the number of seats each party should get
    seats = [0 for i in range(n_parties)]
    # quickly handle winner takes all case, 1 rep per district
    if reps_per_dist == 1:
        seats[np.argmax(votes)] = 1
        return seats
    
    # otherwise, resume calculation
    n_elected = 0
    # Assume there are three representatives per district
    # First round: Award one seat to each party with at least 25% of votes
    for i in range(n_parties):
        if n_elected < n_rep and votes[i] >= 1 / (n_rep + 1):
            seats[i] += 1
            n_elected += 1
            votes[i] -= 1 / (n_rep + 1)
    # Second round: To each party award one seat every 25% of votes
    # At most one party may be awarded seats in this round (three-seat case)
    for i in range(n_parties):
        while n_elected < n_rep and votes[i] >= 1 / (n_rep + 1):
            seats[i] += 1
            n_elected += 1
            votes[i] -= 1 / (n_rep + 1)
    # Third round: Award remaining seats based on ranking of remaining votes
    while n_elected < n_rep:
        p = np.argmax(votes)
        seats[p] += 1
        n_elected += 1
        votes[p] = 0
    return seats
    
def get_all_dist_seats(dist_results, n_rep):
    reps_per_dist = int(243 / len(dist_results))
    # Get the numbers of seats for each party in all districts
    # Step 1: Initialization
    dist_seats = {}
    # Step 2: Calculate numbers of seats district by district
    for d in dist_results:
        dist_seats[d] = get_one_dist_seats(dist_results[d], n_rep, reps_per_dist)
    return dist_seats

def get_total_seats(dist_seats):
    # Get the total number of seats for each party
    # Step 1: Initialization
    n_parties = len(dist_seats[-1])  # number of parties
    total_seats = [0 for i in range(n_parties)]
    # Step 2: Calculate the total number of seats for each party
    for d in dist_seats:
        # d = -1 for the "overall" district, not needed here
        if d != -1:
            for i in range(n_parties):
                total_seats[i] += dist_seats[d][i]
    return total_seats



### Geometry and function for generating random districts

In [38]:
# functions to get points on sides given x
def y_bottom(x):
    return 0
def y_left(x):
    return x * np.sqrt(3)
def y_right(x):
    return np.sqrt(3) * (1000 - x)

# check bounds
def in_bounds(point):
    x, y = point[0], point[1]
    condition1 = y >= y_bottom(x)
    condition2 = y <= y_left(x)
    condition3 = y <= y_right(x)
    return condition1 and condition2 and condition3

def generate_random_centroids(num_districts):
    centroids = np.zeros((num_districts,2))
    for i in range(num_districts):
        while True:
            x = np.random.random() * 1000
            y = np.random.random() * 1000
            proposal = np.array([x,y])
            if in_bounds(proposal):
                centroids[i] = proposal
                break
                
    return centroids

### Newest validity checker

In [71]:
def map_districts_to_voters(districts, voters):
    voters_by_district = []
    for i in range(len(districts)):
        voters_by_district.append([])
    for idx, district in enumerate(districts):
        print(district, "ATTEMPT DISTRICT")
        District = Polygon(district)
        first = True
        for voter in voters:
            voter_loc = Point(voter[0], voter[1])
            if District.contains(voter_loc):
                if first:
                    voters_by_district[idx] = np.array([voter])
                    first = False
                else:
                    voters_by_district[idx] = np.vstack((voters_by_district[idx], voter))
    return voters_by_district

def run_vote(districts, voters):
    for dist in districts:
        dist_voters = map_districts_to_voters(districts, voters)
        reps_per_dist = int(243 / len(districts))
        dist_results = get_dist_results(dist_voters)
        dist_seats = get_all_dist_seats(dist_results, reps_per_dist)
        total_seats = get_total_seats(dist_seats)
    return total_seats

def get_valid_districts_vanilla(population_coords, num_districts):
    centroids, distortion = kmeans(population_coords, num_districts)
    districts = get_districts(centroids) 
    return districts

def get_popular_vote(voters):
    popular_vote = [0 for i in range(len(voters[0]) - 2)]
    for voter in voters:
        prefs = voter[2:]
        fav_party = np.argmax(prefs) + 1
        popular_vote[fav_party - 1] += 1
    return popular_vote / np.sum(popular_vote)

def check_validity2(dist_to_voters, voters):
    num_invalid = 0
    tolerance = 1
    mean_voters = len(voters) / len(dist_to_voters)
    upper_lim = mean_voters * (1 + tolerance)
    lower_lim = mean_voters * (1 - tolerance)
    for dist_pop in dist_to_voters:
        if (len(dist_pop) < lower_lim) or (len(dist_pop) > upper_lim):
            num_invalid += 1
    return num_invalid

def plot_state(centroids, pop_coords):
    fig, ax = plt.subplots(1,1)
    pop_x = pop_coords[ : ,0]
    pop_y = pop_coords[ : ,1]
    centroids_x = centroids[: ,0]
    centroids_y = centroids[: ,1]
    ax.scatter(pop_x, pop_y, color="b", alpha=.2, s=5)
    ax.scatter(centroids_x, centroids_y, color="r", s=8)
    ax.set_title("Current State")
    
def run_experiment(num_generations, reps_per_dist, voters):
    num_districts = int(243 / reps_per_dist)
    for i in range(num_generations):
        thinned_voters = thinner(voters, 1000)
        thinned_Population_coords = thinned_voters[: ,0:2]
        centroids, districts = get_valid_districts(thinned_Population_coords, num_districts)
        result = run_vote(districts, thinned_voters)

        if i == 0:
            results = np.array([result])
        else:
            results = np.vstack((results, np.array(result)))
    return results

### Map information extraction functions

In [40]:
num_population = 333333
num_parties = 2 # default
Population_coords = []
Preferences = []

def extractMapInfo(textfile, test=False):
    with open(textfile) as f:
        first_line = f.readline().split(" ")
        # get num_population
        file_num_population = int(first_line[0])
        if test:
            file_num_population = num_population_test
        # get num_parties
        num_parties = int(first_line[1])
        
        for i in range(file_num_population):
            person = f.readline().rstrip('\n').split(" ")
            Population_coords.append([float(x)  for x in person[:2]])
            Preferences.append([float(pref) for pref in person[2:(2 + num_parties)]])
            
    
    
path = os.getcwd() + "/texas_2_real_data_map.map"
extractMapInfo(path)

### Extract Populations by preference to be used for more sophisticated k-means centroids

In [41]:
Pop_1_coords = []
Pop_2_coords = []
Pop_3_coords = []

def extractSubPopulationCoords():
    party_pref = 1 # default
    
    for i in range(len(Population_coords)):
        for j in range(num_parties):
            # if this preference is the highest
            if Preferences[i][j] == max(Preferences[i]):
                if j == 0:
                    Pop_1_coords.append(Population_coords[i])
                if j == 1:
                    Pop_2_coords.append(Population_coords[i])
                if j == 2:
                    Pop_3_coords.append(Population_coords[i])

extractSubPopulationCoords()             

### Voronoi functions

In [42]:
# Credit for teselating Voronoi in python
# https://gist.github.com/pv/8036995
# https://stackoverflow.com/questions/20515554/colorize-voronoi-diagram/20678647#20678647

def voronoi_finite_polygons_2d(vor, radius=None):
    
    if vor.points.shape[1] != 2:
        raise ValueError("Requires 2D input")

    new_regions = []
    new_vertices = vor.vertices.tolist()

    center = vor.points.mean(axis=0)
    if radius is None:
        radius = vor.points.ptp().max()*2

    # Construct a map containing all ridges for a given point
    all_ridges = {}
    for (p1, p2), (v1, v2) in zip(vor.ridge_points, vor.ridge_vertices):
        all_ridges.setdefault(p1, []).append((p2, v1, v2))
        all_ridges.setdefault(p2, []).append((p1, v1, v2))

    # Reconstruct infinite regions
    for p1, region in enumerate(vor.point_region):
        vertices = vor.regions[region]

        if all(v >= 0 for v in vertices):
            # finite region
            new_regions.append(vertices)
            continue

        # reconstruct a non-finite region
        ridges = all_ridges[p1]
        new_region = [v for v in vertices if v >= 0]

        for p2, v1, v2 in ridges:
            if v2 < 0:
                v1, v2 = v2, v1
            if v1 >= 0:
                # finite ridge: already in the region
                continue

            # Compute the missing endpoint of an infinite ridge

            t = vor.points[p2] - vor.points[p1] # tangent
            t /= np.linalg.norm(t)
            n = np.array([-t[1], t[0]])  # normal

            midpoint = vor.points[[p1, p2]].mean(axis=0)
            direction = np.sign(np.dot(midpoint - center, n)) * n
            far_point = vor.vertices[v2] + direction * radius

            new_region.append(len(new_vertices))
            new_vertices.append(far_point.tolist())

        # sort region counterclockwise
        vs = np.asarray([new_vertices[v] for v in new_region])
        c = vs.mean(axis=0)
        angles = np.arctan2(vs[:,1] - c[1], vs[:,0] - c[0])
        new_region = np.array(new_region)[np.argsort(angles)]

        # finish
        new_regions.append(new_region.tolist())

    return new_regions, np.asarray(new_vertices)

def get_max_sides(districts):
    max_sides = 0
    for poly in districts:
        if len(poly) > max_sides:
            max_sides = len(poly)
    return max_sides

def get_districts_visualize(points):
    # compute Voronoi tesselation
    vor = Voronoi(points)

    # plot
    regions, vertices = voronoi_finite_polygons_2d(vor)
    threeland = Polygon([[0,0], [1000,0], [500, 500 * np.sqrt(3)]])
    districts = []
    
    fig, ax = plt.subplots(1,1, figsize=(12,6))

    # colorize
    for region in regions:
        polygon = vertices[region]
        # Clipping polygon
        poly = Polygon(polygon)
        poly = poly.intersection(threeland)
        polygon = [p for p in poly.exterior.coords]
        if visualize:
            ax.fill(*zip(*polygon), alpha=0.4)
        districts.append(polygon)

    ax.plot(points[:, 0], points[:, 1], 'ko', ms=3)
    ax.axis('equal')
    ax.set_title("Preview of {} Districts".format(len(districts)))
    
    return districts

def get_districts(points):
    # compute Voronoi tesselation
    vor = Voronoi(points)
    regions, vertices = voronoi_finite_polygons_2d(vor)
    threeland = Polygon([[0,0], [1000,0], [500, 500 * np.sqrt(3)]])
    districts = []

    for region in regions:
        polygon = vertices[region]
        # Clipping polygon
        poly = Polygon(polygon)
        poly = poly.intersection(threeland)
        polygon = [p for p in poly.exterior.coords]
        districts.append(polygon)
    
    return districts

### Build file with voters and districts

In [43]:
newFileName = "map" + str(num_parties) + "_reps" + str(NUM_PARTY_REPS) + "_results"
# print (newFileName) # test

# get file ready (don't have districts yet)
def buildNewFile(mapfile):
    read_file = open(mapfile, "r")
    write_file = open(newFileName, "w")
    
    for i in range(num_population +1):
        write_file.write(read_file.readline())
    read_file.close()
    write_file.close()

buildNewFile(path)
num_districts = 243 / NUM_PARTY_REPS
    
def append_districts(name, districts):
    f = open(name, "a+")
    f.write(str(len(districts)) + "\n")
    for poly in districts:
        f.write(str(len(poly)) + " ")
        for vertex in poly:
            x, y = vertex[0], vertex[1]
            f.write(str(x) + " " + str(y) + " ")
        f.write("\n")
    f.close()

### Population thinner

In [44]:
# population is numpy array of shape (num_pop, 2).  x is idx 0 and y 1.
def thinner(population, thinned_length):
    sample = np.random.choice(len(population), thinned_length)
    return population[sample]

### Essential Data Structure Summary

districts is simply a list of polygons.  Each polygon is a list of vertices, each of which is a (x,y) tuple.

voters is a numpy array with shape = (num_voters, 2 + num_parties).  In other words, each voter is an x, y coord followed by their prefs for each party.

k_means_centroids is a numpy array of centroids, shape = (num_centroids, 2)

These 3 objects basically define our complete situation.  Everything we need to move on with analysis rests with algorithmically generating a valid set of these objects.  Once we can get 1 set, I can get 10,000 sets and generate all the voting / gerrymandering data we need.

### Make file / Get preview

In [45]:
Population_coords = np.array(Population_coords)
Preferences = np.array(Preferences)

voters = np.hstack((Population_coords, Preferences))
thinned_voters = thinner(voters, 1000)
thinned_Population_coords = thinned_voters[: ,0:2]



In [46]:
# time-consuming, n-p hard function
#from equal_groups import EqualGroupsKMeans

# egk = EqualGroupsKMeans(n_clusters = 243//NUM_PARTY_REPS, max_iter = 40)
# egk.fit(thinned_Population_coords)
# centroids_equal_pop = egk.cluster_centers_
# len(thinned_Population_coords)

### Equal Groups k-means functions satisfying pop restriction

In [47]:
# repurposed from https://github.com/ndanielsen/Same-Size-K-Means
from equal_groups import EqualGroupsKMeans

# break up pop in 9;
# compute k number of clusters for the people in each smaller triangle
# in proportion to the population there



def get_nine_pops(thinned_pop, visualize=False ): # len(thinned_pop) = 1000
    
    # first, define sub-districts using their centers

    # 9 centers with two coordinates (used to define each of the 9 sub-districts)
    Pops_Nine_Centers = np.zeros((9, 2))
    
    centroid_of_3_land = np.array([500, 500/np.sqrt(3)])

    inner_radius = 2/3 * 500/np.sqrt(3)
    # inner 6 districts surrounding centroid of 3-land
    for i in range(6):
        angle = 2*np.pi * i/6
        direction = np.array([np.sin(angle), np.cos(angle)])
        Pops_Nine_Centers[i] = centroid_of_3_land + inner_radius * direction
    
    outer_radius = inner_radius *2
    for i in range(3):
        angle = 2*np.pi * i/3
        direction = np.array([np.sin(angle), np.cos(angle)])
        Pops_Nine_Centers[i +6] = centroid_of_3_land + outer_radius * direction
    
    # second, find distances between each sub-district center and all the people
    Pop_Distances = np.zeros((len(thinned_pop), 9))
    for i, pops_nine_center_i in enumerate(Pops_Nine_Centers):
        # nice way to subtract xs and ys all at once!
        offsets_i = thinned_pop - pops_nine_center_i
        distances_i = np.sqrt((offsets_i**2).sum(axis = 1))
        # set everything for the ith column
        Pop_Distances[:,i] = distances_i
        
    # third, add person from thinned-pop to sub-district with "center" that it's closest to
    # argmin tells us which of the boxes led to that minimum; one-liner returns index (AKA sub-district number)!
    Pop_Indices = Pop_Distances.argmin(axis = 1)
    
    # fourth, for each district, consolidate belonging points
    Pops_Nine = [None] *9
    
    for i in range(9):
        # boolean corresponding to when i is the same and any one of the pop indices; one-liner!
        belongs_to_district_i = Pop_Indices == i
        # extract only the belonging population
        Pops_Nine[i] = thinned_pop[belongs_to_district_i]

    # plot to see what's going on
    if visualize:
        import matplotlib.pyplot as plt
        plt.plot(Pops_Nine_Centers[:,0], Pops_Nine_Centers[:,1], "kx")
        plt.scatter(thinned_pop[:,0], thinned_pop[:,1], c = Pop_Indices)
        plt.axis("equal")
        plt.show()
    
    return Pops_Nine




def get_k(Pops_Nine, num_reps_per_district):
    num_dist = int(243 / num_reps_per_district)
    Ks_Nine = [None] *len(Pops_Nine)
    
    num_thinned_pop = len(thinned_voters)
    
    # pops_nine_i is index i of Pops_Nine
    for i, pops_nine_i in enumerate(Pops_Nine):
        fractional_population = len(pops_nine_i) / num_thinned_pop
        Ks_Nine[i] = int(num_dist * fractional_population) # the floor
        
    # set the very first k to what is needs to be to ensure correct sum of ks (& add all ks except for very first)
    Ks_Nine[0] = int(num_dist - sum(Ks_Nine[1:]))
    
    return Ks_Nine



def get_nine_pops_centroids( Pops_Nine, Ks_Nine ):
    Centroids_Nine_Consolidated = []

    for i, ks_nine_i in enumerate(Ks_Nine):
        if ks_nine_i == 0:  continue

        egk = EqualGroupsKMeans(n_clusters = ks_nine_i, max_iter = 40)
        egk.fit(Pops_Nine[i])
        centroids = egk.cluster_centers_
        # need to append them one after the other
        Centroids_Nine_Consolidated.append(centroids)

    # concatenate the merged points
    Centroids_Nine_Consolidated = np.concatenate(Centroids_Nine_Consolidated)
    return Centroids_Nine_Consolidated

In [48]:
def get_valid_districts(pop_coords, num_reps_per_district):
    Pops_Nine = get_nine_pops(pop_coords)
    Ks_Nine = get_k(Pops_Nine, num_reps_per_district)
    Centroids_Nine_Consolidated = get_nine_pops_centroids(Pops_Nine, Ks_Nine)
    districts = get_districts(Centroids_Nine_Consolidated)
    return Centroids_Nine_Consolidated, districts

In [49]:
centroids, districts = get_valid_districts(thinned_Population_coords, 3)
dist_to_voters = map_districts_to_voters(districts=districts, voters=thinned_voters)
check_validity2(dist_to_voters=dist_to_voters, voters=thinned_voters)

0

In [72]:
results = run_experiment(num_generations=5, reps_per_dist=3, voters=voters)

3
district from get_valid in run_experiment
[(376.708360649854, 456.2697643276882), (472.83775761708387, 614.5377928165881), (491.04069116481344, 495.15390899051124), (376.708360649854, 456.2697643276882)]



[(491.04069116481344, 495.15390899051124), (472.83775761708387, 614.5377928165881), (566.6902232106538, 401.0209880309094), (491.04069116481344, 495.15390899051124)]



[(566.6902232106538, 401.0209880309094), (376.708360649854, 456.2697643276882), (491.04069116481344, 495.15390899051124), (566.6902232106538, 401.0209880309094)]



[(376.708360649854, 456.2697643276882), (472.83775761708387, 614.5377928165881), (491.04069116481344, 495.15390899051124), (376.708360649854, 456.2697643276882)] ATTEMPT DISTRICT
[(491.04069116481344, 495.15390899051124), (472.83775761708387, 614.5377928165881), (566.6902232106538, 401.0209880309094), (491.04069116481344, 495.15390899051124)] ATTEMPT DISTRICT
[(566.6902232106538, 401.0209880309094), (376.708360649854, 456.2697643276882), (491.0406911648

[(479.5215754982414, 447.24699784184315), (467.39462480157897, 639.7099991622097), (667.2050399448433, 402.9301050833953), (479.5215754982414, 447.24699784184315)] ATTEMPT DISTRICT
[(291.0963690122574, 406.1981078305778), (467.39462480157897, 639.7099991622097), (479.5215754982414, 447.24699784184315), (291.0963690122574, 406.1981078305778)] ATTEMPT DISTRICT


In [73]:
normalized_results = results/243
np.max(normalized_results[: ,0])

0.7818930041152263

In [74]:
get_popular_vote(voters=voters)

array([0.4999055, 0.5000945])